# VacationPy## 
Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port alfred,-33.5906,26.8910,21.77,79,5,3.13,ZA,1701931119
1,1,papatowai,-46.5619,169.4708,12.07,65,100,6.08,NZ,1701931120
2,2,bolotnoye,55.6717,84.3981,-22.04,89,100,3.09,RU,1701931122
3,3,nybro,56.7446,15.9071,1.25,99,100,3.12,SE,1701931123
4,4,punta arenas,-53.1500,-70.9167,7.06,81,0,8.75,CL,1701931011


### Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_1 = city_data_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True, 
    colorbar = True, 
    alpha=0.5,
    xlim=(-150, 160), 
    ylim=(-55, 72), 
    tiles='OSM',
     size = "Humidity",
      scale = 1,
     color = "City"
)

# Display the map
# YOUR CODE HERE
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values

#A max temperature lower than 27 degrees but higher than 21
#Wind speed less than 4.5 m/s
#Zero cloudiness

ideal_city_df = city_data_df.loc[
    (city_data_df["Max Temp"].between(21, 27, inclusive='both')) & 
    (city_data_df["Wind Speed"] <= 4.5) &
    (city_data_df["Cloudiness"] == 0)
]

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
55,55,cabo san lucas,22.8909,-109.9124,23.62,81,0,3.57,MX,1701931193
58,58,kruisfontein,-34.0033,24.7314,22.71,48,0,1.58,ZA,1701931197
90,90,jiujiang,29.7333,115.9833,22.00,28,0,2.89,CN,1701931241
177,177,behbahan,30.5959,50.2417,22.06,29,0,1.10,IR,1701931365
190,190,sur,22.5667,59.5289,25.93,35,0,1.78,OM,1701931385
199,199,bushehr,28.9684,50.8385,24.05,46,0,0.00,IR,1701931398
255,255,pasni,25.2631,63.4710,23.47,47,0,1.47,PK,1701931473
276,276,palanpur,24.1667,72.4333,25.45,36,0,2.34,IN,1701931504
333,333,bredasdorp,-34.5322,20.0403,23.25,56,0,3.16,ZA,1701931582
370,370,al wajh,26.2455,36.4525,21.97,60,0,1.54,SA,1701931883


### Step 3: Create a new DataFrame called hotel_df.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_city_df[['City','Country', 'Lat','Lng','Humidity']].copy()
    
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ''

# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
55,cabo san lucas,MX,22.8909,-109.9124,81,
58,kruisfontein,ZA,-34.0033,24.7314,48,
90,jiujiang,CN,29.7333,115.9833,28,
177,behbahan,IR,30.5959,50.2417,29,
190,sur,OM,22.5667,59.5289,35,
199,bushehr,IR,28.9684,50.8385,46,
255,pasni,PK,25.2631,63.4710,47,
276,palanpur,IN,24.1667,72.4333,36,
333,bredasdorp,ZA,-34.5322,20.0403,56,
370,al wajh,SA,26.2455,36.4525,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000 # YOUR CODE HERE




params = {"categories":"accommodation.hotel",# YOUR CODE HERE
          "apiKey":geoapify_key,
          "limit":1}



# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
        
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}" # YOUR CODE HERE
    params["bias"] = f"proximity:{longitude},{latitude}" # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
               

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params) # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = name_address.json() # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cabo san lucas - nearest hotel: Comfort Rooms
kruisfontein - nearest hotel: No hotel found
jiujiang - nearest hotel: 九江信华建国酒店
behbahan - nearest hotel: هتل بین المللی رضا
sur - nearest hotel: Sur Hotel
bushehr - nearest hotel: هتل رستوران سیراف
pasni - nearest hotel: Juddi Hotel
palanpur - nearest hotel: Hotel Way Wait
bredasdorp - nearest hotel: Victoria Hotel
al wajh - nearest hotel: فندق رالين 2
as suwayq - nearest hotel: No hotel found
moroto - nearest hotel: Shalosa Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
55,cabo san lucas,MX,22.8909,-109.9124,81,Comfort Rooms
58,kruisfontein,ZA,-34.0033,24.7314,48,No hotel found
90,jiujiang,CN,29.7333,115.9833,28,九江信华建国酒店
177,behbahan,IR,30.5959,50.2417,29,هتل بین المللی رضا
190,sur,OM,22.5667,59.5289,35,Sur Hotel
199,bushehr,IR,28.9684,50.8385,46,هتل رستوران سیراف
255,pasni,PK,25.2631,63.4710,47,Juddi Hotel
276,palanpur,IN,24.1667,72.4333,36,Hotel Way Wait
333,bredasdorp,ZA,-34.5322,20.0403,56,Victoria Hotel
370,al wajh,SA,26.2455,36.4525,60,فندق رالين 2


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_2 = hotel_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True, 
    colorbar = True, 
    alpha=0.5,
    xlim=(-150, 160), 
    ylim=(-55, 72), 
    tiles='OSM',
     size = "Humidity",
      scale = 1,
     color = "City",
     hover_cols = ['Lng', 'Lat',"City", "Humidity", "Hotel Name", "Country"]
)

# Display the map
# YOUR CODE HERE
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)